<center> 
    <h1><b>CSC173: Group Activity 1<b></h1>
</center>

<center><i>Members: Febe Belvis, Kervin Paalisbo, Joshua Radz Adlaon</i></center>

This notebook documents this group's Activity 1, where the goal is to implement a neural network from scratch (without using machine learning libraries such as TensorFlow or PyTorch). The neural network will be trained and evaluated on the Breast Cancer dataset from the UCI Machine Learning Repository.

***

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo # to directly import the dataset from its repo


## **Loading and Preprocessing the Dataset**

Before we use train the network, we import the dataset from its repository using the ```ucimlrepo``` library. 

In [50]:
breast_cancer = fetch_ucirepo(id=17) # import the dataset from the ML repo

# extract features and its values/targets
X = breast_cancer.data.features
y = breast_cancer.data.targets

# combine into one DataFrame
df = pd.concat([X, y], axis=1)

We then preprocess the dataset by standardizing the feature names and converting the diagnosis values from ```'M/B'``` to ```'1/0'```.

In [51]:
# lowercase 'diagnosis' column for standardization 
df.rename(columns={df.columns[-1]: 'diagnosis'}, inplace=True)

# convert diagnosis values to binary (M=1, B=0)
y = y['Diagnosis'].map({'M':1,'B':0})
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

df.head()

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,...,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1



## **Feature Selection (using Correlation) and Standardization**
In this step, we identify the two features most correlated with the target ```diagnosis```. <br>We use the **Pearson correlation coefficient**, particularly panda's ```corr``` function to measures the linear relationship between two variables. 

A higher absolute correlation value (closer to 1 or -1) means that changes in the feature are strongly associated with changes in the target, making that feature more useful for prediction.


In [ ]:
corr = df.corr(method='pearson')['diagnosis'].abs().sort_values(ascending=False)
corr.head(5)

diagnosis          1.000000
concave_points3    0.793566
perimeter3         0.782914
concave_points1    0.776614
radius3            0.776454
perimeter1         0.742636
area3              0.733825
radius1            0.730029
area1              0.708984
concavity1         0.696360
Name: diagnosis, dtype: float64


<span style="font-size:12px">*GeeksforGeeks. (2025, July 23). Pearson Correlation Coefficient. GeeksforGeeks. https://www.geeksforgeeks.org/maths/pearson-correlation-coefficient/*</span>
<br><span style="font-size:12px">*GeeksforGeeks. (2025b, July 11). Pandas DataFrame corr() Method. GeeksforGeeks. https://www.geeksforgeeks.org/pandas/python-pandas-dataframe-corr/*</span>




<br><br>Since we're limited to using only 2 input features, we pick the top 2 most correlated features to ```diagnosis```

In [48]:
top_features = corr.index[1:3].tolist()

# extract top 2 correlated features
X_selected = X[top_features]

print("Selected features:", top_features)

Selected features: ['concave_points3', 'perimeter3']
